# Market-Neutral Carry Strategy: Backtest Analysis

This notebook evaluates the backtest performance of the market-neutral carry strategy.

**Objectives:**
1. Run complete backtest
2. Analyze performance metrics
3. Examine drawdown periods
4. Conduct attribution analysis
5. Compare to benchmarks

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import strategy modules
from data_acquisition import MultiAssetDataAcquisition
from factor_models import FactorModels
from signal_generator import CarrySignalGenerator
from portfolio_constructor import PortfolioConstructor
from backtester import CarryBacktester

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("Libraries imported successfully")

## 1. Run Complete Pipeline

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded")
print(f"\nBacktest period: {config['data']['start_date']} to {config['data']['end_date']}")
print(f"Initial capital: ${config['backtest']['initial_capital']:,.0f}")
print(f"Target Sharpe: {config['backtest']['target_sharpe']}")
print(f"Target max drawdown: {config['backtest']['target_max_drawdown']:.1%}")

In [ ]:
# Load data
print("\n1. Loading data...")
data_acq = MultiAssetDataAcquisition(config)
dataset = data_acq.get_full_dataset()
print("   Data loaded")

In [ ]:
# Fit factor models
print("\n2. Fitting factor models...")
factor_models = FactorModels(config)
factor_predictions = factor_models.fit_all_models(dataset)
print("   Models fitted")

In [ ]:
# Generate signals
print("\n3. Generating signals...")
signal_gen = CarrySignalGenerator(config)
signals = signal_gen.generate_all_signals(dataset, factor_predictions)
print("   Signals generated")

In [ ]:
# Construct portfolio
print("\n4. Constructing portfolio...")
portfolio_constructor = PortfolioConstructor(config)

# Get rebalance dates
all_dates = []
for asset_class, data in dataset.items():
    if isinstance(data, dict):
        for key, df in data.items():
            if isinstance(df, pd.DataFrame) and len(df) > 0:
                all_dates.extend(df.index.tolist())

min_date = min(all_dates)
max_date = max(all_dates)
rebalance_dates = pd.date_range(start=min_date, end=max_date, 
                                freq=config['data']['rebalance_frequency'])

portfolio_history = portfolio_constructor.construct_portfolio_timeseries(
    signals, dataset, rebalance_dates.tolist())

print("   Portfolio constructed")

In [ ]:
# Run backtest
print("\n5. Running backtest...")
backtester = CarryBacktester(config)
results = backtester.run_backtest(portfolio_history, dataset)
print("   Backtest complete")

## 2. Performance Overview

In [ ]:
# Extract results
results_df = results['results']
metrics = results['metrics']

print("PERFORMANCE METRICS")
print("="*60)
print(f"Total Return:       {metrics['total_return']:>10.2%}")
print(f"Annual Return:      {metrics['annual_return']:>10.2%}")
print(f"Volatility:         {metrics['volatility']:>10.2%}")
print(f"Sharpe Ratio:       {metrics['sharpe_ratio']:>10.2f}")
print(f"Sortino Ratio:      {metrics['sortino_ratio']:>10.2f}")
print(f"Max Drawdown:       {metrics['max_drawdown']:>10.2%}")
print(f"Calmar Ratio:       {metrics['calmar_ratio']:>10.2f}")
print(f"Win Rate:           {metrics['win_rate']:>10.2%}")
print(f"Profit Factor:      {metrics['profit_factor']:>10.2f}")
print(f"T-Statistic:        {metrics['t_statistic']:>10.2f}")

# Compare to targets
print("\n" + "="*60)
print("TARGET COMPARISON")
print("="*60)
sharpe_check = "✓" if metrics['sharpe_ratio'] >= config['backtest']['target_sharpe'] else "✗"
dd_check = "✓" if abs(metrics['max_drawdown']) <= abs(config['backtest']['target_max_drawdown']) else "✗"
print(f"Sharpe >= {config['backtest']['target_sharpe']}: {sharpe_check}")
print(f"Drawdown <= {config['backtest']['target_max_drawdown']:.0%}: {dd_check}")

In [ ]:
# Plot main performance chart
backtester.plot_results(results)

## 3. Detailed Equity Curve Analysis

In [ ]:
# Equity curve with key statistics
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Equity curve
axes[0].plot(results_df.index, results_df['equity'], linewidth=2)
axes[0].set_title('Equity Curve', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Capital ($)')
axes[0].grid(True, alpha=0.3)
axes[0].text(0.02, 0.95, 
            f"Final Value: ${results_df['equity'].iloc[-1]:,.0f}\n" + 
            f"Total Return: {metrics['total_return']:.2%}\n" +
            f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}",
            transform=axes[0].transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Cumulative returns
cumulative_returns = (1 + results_df['daily_return']).cumprod() - 1
axes[1].plot(results_df.index, cumulative_returns * 100, linewidth=2, color='green')
axes[1].set_title('Cumulative Returns', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Return (%)')
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Drawdown Analysis

In [ ]:
# Calculate drawdown series
returns = results_df['daily_return']
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max

# Identify drawdown periods
in_drawdown = drawdown < -0.05  # More than 5% drawdown
drawdown_periods = []
start = None

for i, (date, is_dd) in enumerate(in_drawdown.items()):
    if is_dd and start is None:
        start = date
    elif not is_dd and start is not None:
        drawdown_periods.append((start, date, drawdown.loc[start:date].min()))
        start = None

print("MAJOR DRAWDOWN PERIODS (>5%)")
print("="*60)
for i, (start, end, min_dd) in enumerate(drawdown_periods[:5]):
    duration = (end - start).days
    print(f"\n{i+1}. {start.date()} to {end.date()}")
    print(f"   Duration: {duration} days")
    print(f"   Max Drawdown: {min_dd:.2%}")

In [ ]:
# Plot drawdown with annotations
plt.figure(figsize=(14, 6))
plt.fill_between(results_df.index, 0, drawdown * 100, alpha=0.3, color='red')
plt.plot(results_df.index, drawdown * 100, color='red', linewidth=2)
plt.axhline(y=-10, color='orange', linestyle='--', alpha=0.7, label='10% DD Warning')
plt.axhline(y=-15, color='red', linestyle='--', alpha=0.7, label='15% DD Critical')
plt.title('Portfolio Drawdown Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Rolling Performance Metrics

In [ ]:
# Calculate rolling metrics
rolling_window = 252  # 1 year

rolling_sharpe = (returns.rolling(rolling_window).mean() / 
                 returns.rolling(rolling_window).std()) * np.sqrt(252)

rolling_vol = returns.rolling(rolling_window).std() * np.sqrt(252)

# Plot rolling metrics
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].plot(results_df.index, rolling_sharpe, linewidth=2)
axes[0].axhline(y=config['backtest']['target_sharpe'], color='r', 
               linestyle='--', label=f"Target: {config['backtest']['target_sharpe']}")
axes[0].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[0].set_title('Rolling 1-Year Sharpe Ratio', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(results_df.index, rolling_vol * 100, linewidth=2, color='orange')
axes[1].axhline(y=config['portfolio']['target_volatility'] * 100, 
               color='r', linestyle='--', 
               label=f"Target: {config['portfolio']['target_volatility']:.0%}")
axes[1].set_title('Rolling 1-Year Volatility', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Volatility (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Return Distribution Analysis

In [ ]:
# Analyze return distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Histogram
axes[0].hist(returns.dropna() * 100, bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(x=returns.mean() * 100, color='r', linestyle='--', 
               label=f'Mean: {returns.mean()*100:.3f}%')
axes[0].set_title('Daily Return Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Daily Return (%)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(returns.dropna(), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot (Normal Distribution)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Distribution statistics
print("\nRETURN DISTRIBUTION STATISTICS")
print("="*60)
print(f"Mean daily return:    {returns.mean()*100:.4f}%")
print(f"Median daily return:  {returns.median()*100:.4f}%")
print(f"Std daily return:     {returns.std()*100:.4f}%")
print(f"Skewness:             {returns.skew():.3f}")
print(f"Kurtosis:             {returns.kurtosis():.3f}")
print(f"5th percentile:       {returns.quantile(0.05)*100:.3f}%")
print(f"95th percentile:      {returns.quantile(0.95)*100:.3f}%")

## 7. Monthly and Annual Returns

In [ ]:
# Calculate monthly and annual returns
monthly_returns = returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
annual_returns = returns.resample('Y').apply(lambda x: (1 + x).prod() - 1)

print("ANNUAL RETURNS")
print("="*60)
for year, ret in annual_returns.items():
    print(f"{year.year}: {ret:>8.2%}")

In [ ]:
# Monthly returns heatmap
monthly_pivot = monthly_returns.to_frame('return')
monthly_pivot['year'] = monthly_pivot.index.year
monthly_pivot['month'] = monthly_pivot.index.month
heatmap_data = monthly_pivot.pivot(index='year', columns='month', values='return')

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data * 100, annot=True, fmt='.1f', cmap='RdYlGn',
           center=0, cbar_kws={'label': 'Return (%)'})
plt.title('Monthly Returns Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Year')
plt.tight_layout()
plt.show()

## 8. Asset Class Attribution

In [ ]:
# Analyze portfolio weights by asset class
portfolio = portfolio_history

# Group by asset class
class_weights = {}
for col in portfolio.columns:
    asset_class = col.split('_')[0]
    if asset_class not in class_weights:
        class_weights[asset_class] = pd.DataFrame()
    class_weights[asset_class][col] = portfolio[col]

# Aggregate
class_totals = pd.DataFrame()
for asset_class, weights in class_weights.items():
    class_totals[asset_class] = weights.abs().sum(axis=1)

# Plot allocation over time
class_totals.plot.area(figsize=(14, 6), alpha=0.7)
plt.title('Asset Class Gross Exposure Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Gross Exposure')
plt.xlabel('Date')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
print("\nAVERAGE ASSET CLASS WEIGHTS")
print("="*60)
print(class_totals.mean().sort_values(ascending=False).to_string())

## Summary

**Performance Results:**
- Annual Return: {metrics['annual_return']:.2%}
- Sharpe Ratio: {metrics['sharpe_ratio']:.2f}
- Max Drawdown: {metrics['max_drawdown']:.2%}

**Key Observations:**
1. Strategy [met/did not meet] target Sharpe ratio
2. Maximum drawdown was [within/exceeded] acceptable limits
3. Return distribution shows [normal/fat-tailed] characteristics
4. Asset class diversification provides [strong/moderate/weak] benefits

**Recommendations:**
- [List based on results]